In [5]:
from collections import Counter
from datasets import load_dataset,load_from_disk
dataset=load_from_disk('codeforces_cot_filtered_truncated')
dataset=dataset.filter(lambda ex:ex['language'] in ['python','cpp'])
langs = [ex["language"] for ex in dataset]
print(Counter(langs))


Filter:   0%|          | 0/54125 [00:00<?, ? examples/s]

Counter({'cpp': 42192, 'python': 11915})


In [8]:
dataset.save_to_disk('codeforces_cot_filtered_truncated_v1')

Saving the dataset (0/7 shards):   0%|          | 0/54107 [00:00<?, ? examples/s]

In [27]:
from collections import Counter
from datasets import load_dataset,load_from_disk
dataset=load_from_disk('new_dataset_annotated_train')
dataset=dataset.filter(lambda ex:ex['dup_top_score']>0.8)
dataset.save_to_disk('codeforces_cot_filtered_truncated_matched_train_v1')

dataset=load_from_disk('new_dataset_annotated_test')
dataset=dataset.filter(lambda ex:ex['dup_top_score']>0.8)
dataset.save_to_disk('codeforces_cot_filtered_truncated_matched_val_v1')

Saving the dataset (0/2 shards):   0%|          | 0/16316 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2068 [00:00<?, ? examples/s]

In [24]:
dataset=dataset.filter(lambda ex:ex['dup_top_score']>0.8)


Filter:   0%|          | 0/54107 [00:00<?, ? examples/s]

In [ ]:
load_dataset("haj1r/sphinxnautics-codeforces-cot-v3")['train']

FileNotFoundError: Directory haj1r/sphinxnautics-codeforces-cot-v3 not found

In [ ]:
from datasets import load_dataset

dataset = load_dataset("haj1r/sphinxnautics-codeforces-cot-v3")



In [45]:
from datasets import load_from_disk, DatasetDict

dataset_combined = DatasetDict({
    "train": load_from_disk("codeforces_cot_filtered_truncated_matched_train_v1"),
    "validation": load_from_disk("codeforces_cot_filtered_truncated_matched_test_v1")
})


In [48]:
ds = load_from_disk("codeforces_cot_filtered_truncated_matched_train_v1")
ds.to_json("train.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

735679355

In [ ]:
dataset_combined.save_to_disk('codeforces_cot_filtered_32B-v2')

Saving the dataset (0/2 shards):   0%|          | 0/16316 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2068 [00:00<?, ? examples/s]

In [25]:
dataset.save_to_disk('codeforces_cot_filtered_truncated_matched_test_v1')

Saving the dataset (0/1 shards):   0%|          | 0/2068 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer
model_name='Qwen/Qwen2.5-7B-Instruct'
chkpoint_path="checkpoints4/Qwen2.5-7B-Instruct/codeforces_cot_filtered_truncated_matched_v1/epoch_1/model_step_500.pt"
chkpoint_path="chekpoint-rl/sphinxnautics-7b/haj1r/sphinxnautics-codeforces-cot-v3/epoch_0/model_step_500.pt"
chkpoint_path="checkpoint-rl-revkl/sphinxnautics-7b/haj1r/sphinxnautics-codeforces-cot-v3/epoch_0/model_step_400.pt"
chkpoint_path="checkpoint-rl-revkl/sphinxnautics-7b/haj1r/sphinxnautics-codeforces-cot-v3/epoch_0/model_step_1800.pt"
#chkpoint_path="checkpoint-distill/sphinxnautics-7b/haj1r/sphinxnautics-codeforces-cot-v3/epoch_0/model_step_700.pt"
#chkpoint_path="checkpoints5/Qwen2.5-7B-Instruct/haj1r/sphinxnautics-codeforces-cot-v3/epoch_2/model_step_1000.pt"
ACCESS_TOKEN = os.getenv("HUGGING_FACE_HUB_TOKEN")
model = AutoModelForCausalLM.from_pretrained(model_name,token=access_token,torch_dtype=torch.bfloat16,attn_implementation="flash_attention_2",device_map='cpu')
ckpt = torch.load(chkpoint_path, map_location='cpu')
model.load_state_dict(ckpt, strict=False)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=access_token)
if tokenizer.pad_token_id is None:
        # Add a new unique padding token
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

dataset_name=chkpoint_path.split('/')[2]
if 'rl' in chkpoint_path.split('/')[0]:
        dataset_name+='_RL'
else:
        dataset_name+='_DL'
if 'rev' in chkpoint_path.split('/')[0]:        
        dataset_name+='rev'

model_name=chkpoint_path.split('/')[1]
epoch_num=chkpoint_path.split('/')[-2].split('_')[-1]
start_step=chkpoint_path.split('/')[-1].split('_')[-1].split('.')[0]
save_dir=f"{model_name.split('/')[-1]}_epoch_{epoch_num}_step_{start_step}_ft_{dataset_name}"
print(save_dir,model.device)


OSError: There was a specific connection error when trying to load Qwen/Qwen2.5-7B-Instruct:
401 Client Error: Unauthorized for url: https://huggingface.co/Qwen/Qwen2.5-7B-Instruct/resolve/main/config.json (Request ID: Root=1-68879820-271cf3d97eb442125cea930f;fb15eaf5-127f-48a1-8e31-15e5e9d79e8a)

Invalid credentials in Authorization header

In [ ]:
save_dir

'sphinxnautics-7b_epoch_0_step_700_ft_haj1r'

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [61]:
import importlib.util

use_flash = importlib.util.find_spec("flash_attn") is not None


In [64]:
use_flash 

True

In [104]:
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/tokenizer_config.json',
 'sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/special_tokens_map.json',
 'sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/chat_template.jinja',
 'sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/vocab.json',
 'sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/merges.txt',
 'sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/added_tokens.json',
 'sphinxnautics-7b_epoch_0_step_700_ft_haj1r_DL/tokenizer.json')